# Using the ORM to store several boreholes

Imports

In [7]:
from os import remove
from utils.io import boreholes_from_files, striplog_from_text
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [8]:
remove('tmp_files/tfe_orm_db.db')

Create the db engine and the database structure

In [9]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [10]:
Base.metadata.create_all(engine)

2021-02-20 14:26:31,416 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-20 14:26:31,418 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:26:31,420 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-20 14:26:31,421 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:26:31,423 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-02-20 14:26:31,424 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:26:31,425 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-02-20 14:26:31,425 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:26:31,426 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-02-20 14:26:31,427 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:26:31,428 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-02-20 14:26:31,429 INFO sqlalchemy.engine.base.Engine ()
2021-02-20

### Create the objects from las files

In [11]:
lexicon = Lexicon.default()

In [12]:
borehole_dict = {'F1':ROOT_DIR+'/data/test.las'}#, 'F2':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [13]:
boreholes, components= boreholes_from_files(borehole_dict, verbose=0)
len(boreholes)

4

Create the session

In [9]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [10]:
p = Project(session)
p.add_components(components)

2021-02-20 14:00:15,528 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:00:15,531 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-20 14:00:15,532 INFO sqlalchemy.engine.base.Engine ()
2021-02-20 14:00:15,537 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-02-20 14:00:15,537 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-02-20 14:00:15,539 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-20 14:00:15,543 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:00:15,544 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-20 14:00:15,544 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [11]:
for bh in boreholes:
    p.add_borehole(bh)

2021-02-20 14:00:16,104 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2021-02-20 14:00:16,106 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-02-20 14:00:16,109 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-02-20 14:00:16,110 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.4764705882

2021-02-20 14:00:16,164 INFO sqlalchemy.engine.base.Engine (14,)
2021-02-20 14:00:16,165 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,166 INFO sqlalchemy.engine.base.Engine (15,)
2021-02-20 14:00:16,170 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,170 INFO sqlalchemy.engine.base.Engine (16,)
2021-02-20 14:00:16,172 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-20 14:00:16,225 INFO sqlalchemy.engine.base.Engine (35,)
2021-02-20 14:00:16,228 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,229 INFO sqlalchemy.engine.base.Engine (36,)
2021-02-20 14:00:16,230 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,231 INFO sqlalchemy.engine.base.Engine (37,)
2021-02-20 14:00:16,236 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-20 14:00:16,336 INFO sqlalchemy.engine.base.Engine (52,)
2021-02-20 14:00:16,337 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,338 INFO sqlalchemy.engine.base.Engine (53,)
2021-02-20 14:00:16,340 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,341 INFO sqlalchemy.engine.base.Engine (54,)
2021-02-20 14:00:16,342 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-20 14:00:16,424 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,424 INFO sqlalchemy.engine.base.Engine (67,)
2021-02-20 14:00:16,428 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,428 INFO sqlalchemy.engine.base.Engine (68,)
2021-02-20 14:00:16,430 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Position

2021-02-20 14:00:16,468 INFO sqlalchemy.engine.base.Engine (87,)
2021-02-20 14:00:16,470 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,471 INFO sqlalchemy.engine.base.Engine (88,)
2021-02-20 14:00:16,472 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,472 INFO sqlalchemy.engine.base.Engine (89,)
2021-02-20 14:00:16,474 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-20 14:00:16,548 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2021-02-20 14:00:16,549 INFO sqlalchemy.engine.base.Engine ('F4',)
2021-02-20 14:00:16,551 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-02-20 14:00:16,552 INFO sqlalchemy.engine.base.Engine ((53, 'F4', 0, 'Anhydrite', 106, 107), (54, 'F4', 1, 'Sandstone, grey, vf-f', 108, 109), (55, 'F4', 2, 'Anhydrite', 110, 111), (56, 'F4', 3, 'Dolomite', 112, 113), (57, 'F4', 4, 'Anhydrite', 114, 115), (58, 'F4', 5, 'Sandstone, grey, vf-f', 116, 117), (59, 'F4', 6, 'Siltstone, red', 118, 119), (60, 'F4', 7, 'Dolomite', 120, 121)  ... displaying 10 of 25 total bound parameter sets ...  (76, 'F4', 23, 'Limestone', 152, 153), (77, 'F4', 24, 'Volcanic', 154, 155))
2021-02-20 14:00:16,553 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-20 14:00:16,557 INFO sqlalchemy.engine.base.Engine BEGIN (implici

2021-02-20 14:00:16,600 INFO sqlalchemy.engine.base.Engine (122,)
2021-02-20 14:00:16,601 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,602 INFO sqlalchemy.engine.base.Engine (123,)
2021-02-20 14:00:16,603 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,604 INFO sqlalchemy.engine.base.Engine (124,)
2021-02-20 14:00:16,605 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

2021-02-20 14:00:16,652 INFO sqlalchemy.engine.base.Engine (143,)
2021-02-20 14:00:16,655 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,655 INFO sqlalchemy.engine.base.Engine (144,)
2021-02-20 14:00:16,656 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-20 14:00:16,657 INFO sqlalchemy.engine.base.Engine (145,)
2021-02-20 14:00:16,659 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Pos

In [12]:
p.boreholes[0].id

'F1'

In [13]:
p.commit()

2021-02-20 14:00:16,731 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
p.boreholes[0].intervals[0].description

2021-02-20 14:00:16,736 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-20 14:00:16,737 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-02-20 14:00:16,737 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-02-20 14:00:16,738 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-20 14:00:16,739 INFO sqlalchemy.engine.base.Engine ('F1',)


'Anhydrite'

In [15]:
session.close()

2021-02-20 14:00:16,873 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [16]:
p.plot3d(x3d=True)